## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd

In [2]:
credited = pd.read_csv('/datasets/data.csv')

In [3]:
credited.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


    Мы можем заметить следующие вещи - столбец 'days_employed' содержит отрицательные значения, что выглядит странно для значений трудового стажа в днях, но сами значения, взятые по модулю будут выглядеть правдоподобно. 
    Также тип данных столбцов 'days_employed' и 'total_income' подразумевает излишнюю точность, тип следует заменить на целочисленный. 
    Значения столбца *education* написаны в разных регистрах.

In [4]:
credited.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


    В столбце *children* присутствуют отрицательные значения, скорее всего, это просто ошибка ввода и мы должны исправить его на значение по модулю. Среднее значение столбца *days_employed* - 172 года, значит есть некорректные значения в записях.
    В столбце *dob_years* присутствуют нулевые значения возраста, скорее всего, они не заполнены.
    

In [5]:
credited.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Значения NAN в столбцах *days_employed* и *total_income*, в остальных - все в порядке. Следует проверить, относятся ли эти пропуски к одной категории занятости или к нескольким. Возможно, они не заполнены случайно. Также видно, что у столбцов *days_employed* и *total_income* тип - *float64*, следует заменить его на целочисленный, что мы заметили ранее.

### Вывод

Каждая строка таблицы содержит информацию о кредитозаемщике банка, характеризующую его социальный статус, а также - информацию о кредитной истории заемщика, доходе, цели получения кредита. 
Проблемы, которые нужно решать: провести предобработку данных, а именно: обработать пропуски и некорректные значения, заменить тип данных у столбцов, там где это нужно, выделить леммы в значениях столбца с целями получения кредита, категоризировать данные. 
Провести исследование о влиянии семейного положения и количества детей клиента на факт погашения кредита в срок.

### Шаг 2. Предобработка данных

### Обработка пропусков

Проверим, в каких полях есть пропуски.

In [6]:
credited.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Это поля *days_employed* и *total_income*. Заменим все значения этих полей на значения по модулю, как планировали ранее.

In [7]:
credited[['days_employed', 'total_income']] = credited[['days_employed', 'total_income']].abs()

Проверим у какого количества клиентов по категориям пропуски в столбце *total_income*:

In [8]:
report = credited.copy()
report['null_income'] = report['total_income'].isna()
report.groupby('income_type').agg({'total_income':'count','null_income':'sum'}).astype(int)

,total_income,null_income
income_type,,
безработный,2,0
в декрете,1,0
госслужащий,1312,147
компаньон,4577,508
пенсионер,3443,413
предприниматель,1,1
сотрудник,10014,1105
студент,1,0


Так как пропуски присутствуют у различных категорий занятости, заполним пропуски NaN в датафрейме credited: в *total_income* заменим NaN на среднее значение *income_type* в каждой группе. Так как значения пропусков в столбце *days_employed* не играют важной роли для поставленных в проекте задач, для удобства заполним их также средними значениями по каждой группе соответственно. 

In [9]:
credited['total_income'] = credited.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))
credited['days_employed'] = credited.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.mean()))
credited.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [10]:
credited.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Посмотрим, какие отрицательные значения содержатся в столбце *children*.

In [11]:
print(credited[credited['children'] < 0].groupby('children')['children'].count())

children
-1    47
Name: children, dtype: int64


Исправим отрицательные значения столбца *children* и переведем в нижний регистр значения столбца *education*.

In [12]:
credited['children'] = credited['children'].replace(-1, 1)
credited['education'] = credited['education'].str.lower()

Для клиентов, у которых значения столбца *dob_years* равны нулю, заполним их средними значениями по категориям.

In [13]:
credited['dob_years'] = credited.groupby('income_type')['dob_years'].transform(lambda x: x.replace(0, int(x.mean())))

### Вывод

In [16]:
 credited['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [17]:
credited['children'] = credited['children'].replace(20, 2)

Обработали пропуски и нестандартные значения.

### Замена типа данных

Значения столбцов *days_employed* и *total_income* (тип - *float64*) заменяем на целочисленный.

In [18]:
credited['days_employed'] = credited['days_employed'].astype('int')
credited['total_income'] = credited['total_income'].astype('int')

Проверим результат.

In [19]:
credited.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

### Вывод

Для замены типа данных воспользовались методом astype() с аргументом int.

### Обработка дубликатов

Посчитаем суммарное количество по дубликатов в датафрейме.

In [20]:
credited.duplicated().sum()

71

Посмотрим эти дубликаты. Записи одинаковые, а id - разные, возможно, это ошибочное дублирующее занесение данных.

In [21]:
credited[credited.duplicated(keep=False)].sort_values(by=['total_income', 'days_employed', 'dob_years'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17787,0,365003,54,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции с жильем
21415,0,365003,54,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции с жильем
3344,0,365003,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции со своей недвижимостью
9627,0,365003,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции со своей недвижимостью
13300,0,365003,56,среднее,1,женат / замужем,0,F,пенсионер,0,137127,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
19369,0,2111,45,среднее,1,гражданский брак,1,F,компаньон,0,202417,свадьба
9920,0,2111,51,среднее,1,гражданский брак,1,F,компаньон,0,202417,на проведение свадьбы
15991,0,2111,51,среднее,1,гражданский брак,1,F,компаньон,0,202417,на проведение свадьбы
2254,0,2111,54,высшее,0,женат / замужем,0,M,компаньон,0,202417,операции с коммерческой недвижимостью


Удалим дубликаты.

In [22]:
credited = credited.drop_duplicates()

Проверим результат.

In [23]:
credited.duplicated().sum()

0

### Вывод

Избавились от дубликатов. Использовали метод duplicates(). В сочетании с методом sum() он возвращает количество дубликатов.

### Лемматизация

Рассмотрим столбец *purpose* нашего датафрейма. В нем описаны различные цели взятия кредита, причем зачастую одна и та же цель сформулирована по-разному. Найдем уникальные значения этого столбца.

In [24]:
print(credited['purpose'].value_counts())

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Лемматизируем полученные значения методом lemmatize() с помощью библиотеки pymystem3. Пройдем в цикле по каждому элементу *credited['purpose']* и применим к нему метод, в итоге нам будет возвращена подходящая форма иходной части текста из словарей библиотеки. Методом extend() соберем значения каждого шага в список.

In [25]:
from pymystem3 import Mystem
m = Mystem()
lemmas = []
for element in credited['purpose']:
    lemma = m.lemmatize(element)
    lemmas.extend(lemma)
from collections import Counter
lemmas = Counter(lemmas)
print(Counter(lemmas))

Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


Теперь отсортируем значения полученного списка по рейтингу включений *pair[1]* по убыванию.

In [26]:
sorted(lemmas.items(), key = lambda pair: pair[1], reverse=True)

[(' ', 33570),
 ('\n', 21454),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

Возьмем для нашего списка уникальных значений 5 из топа, для этого исключим два первых элемента, элементы с длиной короче 4, пусть их будет 8, чтобы последним захваченным значением было *строительство*.

In [27]:
purposes_list = [k for k in sorted(lemmas, key=lemmas.get, reverse=True) if len(k) > 4 if k != ' ' if k != '\n'][0:8]
print(purposes_list)

['недвижимость', 'покупка', 'жилье', 'автомобиль', 'образование', 'операция', 'свадьба', 'строительство']


Оценим получившиеся значения по смыслу, элемент *покупка* может иметь отношение к различным целям, в отдекльности смысла не несет, *строительство* == *жилье*, *операция* ==  *жилье*, поэтому - исключаем.

In [28]:
purposes_list.remove('покупка') 
purposes_list.remove('строительство')
purposes_list.remove('операция')
purposes_list.remove('жилье')
print(purposes_list)

['недвижимость', 'автомобиль', 'образование', 'свадьба']


Присвоим функцией enumerate в цикле номера отобранных целей начиная с единицы.

In [29]:
purposes_list = {value:counter for counter,value in enumerate(purposes_list,1)}
print(purposes_list)

{'недвижимость': 1, 'автомобиль': 2, 'образование': 3, 'свадьба': 4}


### Вывод

Таким образом, мы лемматизировали столбец целей, создали словарь со значениями целей и пронумеровали их.

### Категоризация данных

In [30]:
credited['purpose_category'] = 0
for row in range(len(credited)):
    for purpose in purposes_list:
        if purpose in credited.iloc[row, 11]:
            credited.iloc[row, 12] = purposes_list[purpose]
        
credited.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,0
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,0
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,3
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,0
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,0
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,0
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,3
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,0
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,0


In [31]:
def income_cat(row):
    if row['total_income'] <= 50000:
        return 'низкий'
    elif 50000 < row['total_income'] <= 120000:
        return 'средний'
    else: 
        return 'высокий'

    
def days_employed_cat(row):
    if row['days_employed'] <= 3652:
        return 'стаж до 10 лет'
    elif 3652 < row['days_employed'] <= 6904:
        return 'стаж 10-30 лет'
    else:
        return 'стаж более 30 лет'


def dob_years_cat(row):
    if row['dob_years'] < 30:
        return 'до 30 лет'
    elif 30 <= row['dob_years'] < 45:
        return '30-45 лет'
    elif 45 <= row['dob_years'] < 65:
        return '45-65 лет'
    else:
        return 'старше 65 лет'

    
def children_cat(row):
    if row['children'] == 0:
        return 'нет детей'
    elif 1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'

In [32]:
credited['income_cat'] = credited.apply(income_cat, axis=1)
credited['days_employed_cat'] = credited.apply(days_employed_cat, axis=1)
credited['dob_years_cat'] = credited.apply(dob_years_cat, axis=1)
credited['children_cat'] = credited.apply(children_cat, axis=1)
credited.head

<bound method NDFrame.head of        children  days_employed  dob_years education  education_id  \
0             1           8437         42    высшее             0   
1             1           4024         36   среднее             1   
2             0           5623         33   среднее             1   
3             3           4124         32   среднее             1   
4             0         340266         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1           4529         43   среднее             1   
21521         0         343937         67   среднее             1   
21522         1           2113         38   среднее             1   
21523         3           3112         38   среднее             1   
21524         2           1984         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1

### Вывод

In [33]:
print(credited['income_cat'].value_counts())
print(credited['days_employed_cat'].value_counts())
print(credited['dob_years_cat'].value_counts())
print(credited['children_cat'].value_counts())

высокий    14609
средний     6473
низкий       372
Name: income_cat, dtype: int64
стаж до 10 лет       14478
стаж более 30 лет     4724
стаж 10-30 лет        2252
Name: days_employed_cat, dtype: int64
45-65 лет        8771
30-45 лет        8608
до 30 лет        3180
старше 65 лет     895
Name: dob_years_cat, dtype: int64
нет детей      14091
1-2 ребенка     6983
многодетные      380
Name: children_cat, dtype: int64


Категоризировали датафрейм по уровню зарплаты, стажу работы, количеству детей.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [34]:
report = credited.pivot_table(index = 'children_cat', columns = 'debt', values = 'gender', aggfunc = 'count')
report.columns=['no_debt','debt']
report['%']=report['debt']/(report['debt']+report['no_debt'])*100
report

,no_debt,debt,%
children_cat,,,
1-2 ребенка,6336,647,9.265359
многодетные,349,31,8.157895
нет детей,13028,1063,7.543822


In [35]:
print(credited.groupby('children_cat')['debt'].mean().sort_values(ascending=False))
print(credited.groupby('family_status')['debt'].mean().sort_values(ascending=False))
print(credited.groupby('income_cat')['debt'].mean().sort_values(ascending=False))
print(credited.groupby('purpose_category')['debt'].mean().sort_values(ascending=False))

purpose_category
2    0.092593
3    0.087440
4    0.080910
0    0.079961
1    0.076983
Name: debt, dtype: float64

In [36]:
# credited.groupby('children_cat')['debt'].mean().sort_values(ascending=False)

### Вывод

Заемщики, не имеющие детей, менее склонны к просрочке по выплатам кредита.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [39]:
report = credited.pivot_table(index = 'family_status', columns = 'debt', values = 'gender', aggfunc = 'count')
report.columns=['no_debt','debt']
report['%']=report['debt']/(report['debt']+report['no_debt'])*100
report

,no_debt,debt,%
family_status,,,
Не женат / не замужем,2536,274,9.750890
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343
гражданский брак,3763,388,9.347145
женат / замужем,11408,931,7.545182


### Вывод

Заемщики, не состоящие в официальном браке, более склонны к просрочке по выплатам кредита.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [40]:
report = credited.pivot_table(index = 'income_cat', columns = 'debt', values = 'gender', aggfunc = 'count')
report.columns=['no_debt','debt']
report['%']=report['debt']/(report['debt']+report['no_debt'])*100
report

,no_debt,debt,%
income_cat,,,
высокий,13419,1190,8.145664
низкий,349,23,6.182796
средний,5945,528,8.156960


### Вывод

Заемщики с уровнем дохода < 50000р. менее склонны нарушать обязательства по выплатам кредита.

- Как разные цели кредита влияют на его возврат в срок?

In [41]:
report = credited.pivot_table(index = 'purpose_category', columns = 'debt', values = 'gender', aggfunc = 'count')
report.columns=['no_debt','debt']
report['%']=report['debt']/(report['debt']+report['no_debt'])*100
print(report)
print(purposes_list)

                  no_debt  debt         %
purpose_category                         
0                   13255  1152  7.996113
1                    2386   199  7.698259
2                     882    90  9.259259
3                    2463   236  8.743979
4                     727    64  8.091024
{'недвижимость': 1, 'автомобиль': 2, 'образование': 3, 'свадьба': 4}


### Вывод

Заемщики, кредитующиеся с целью улучшения жилищных условий, менее склонны нарушать обязательства по выплатам кредита в срок.

### Шаг 4. Общий вывод

Семейное положение и количество детей клиента влияет на факт погашения кредита в срок.
По результатам проведенного исследования можно сказать, что самыми ответственными заемщиками можно назвать категорию людей, не имеющих детей, состоящих в официальном браке, имеющих доход менее 50000р, кредитующихся с целью улучшения жилищных условий.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.